In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from reservoirpy import nodes, datasets
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random
import copy
import networkx as nx
import reservoirpy as rpy
rpy.verbosity(0)

# Assuming the helper functions from the provided scripts are imported correctly:
import hierarchical_genome_mutations_operations as hgmo
import evolution_step_functions as esf
import genome_analysis as ga
import genome_transcription as gt
import helper_functions as hf
import hierarchical_genomes as hg

In [2]:
# Define other required parameters
population_size = 50
n_generations = 200
genome_length = 100
n_timesteps = 2000

# Initialize your dataset
X = datasets.mackey_glass(n_timesteps=n_timesteps, sample_len=2000)
Y = np.roll(X, -1)

# Split the dataset
train_end = int(len(X) * 0.7)
test_start = train_end + 1
X_train, Y_train = X[:train_end], Y[:train_end]
X_test, Y_test = X[test_start:], Y[test_start:]

In [3]:
def initialize_genome(genome_length):
    genome = []
    for _ in range(genome_length):
        input_node = np.random.randint(0, 10)  # Example range, adjust as needed
        output_node = np.random.randint(0, 10)  # Example range, adjust as needed
        weight = np.random.uniform(-1, 1)  # Weight range -1 to 1, adjust as needed
        genome.append((input_node, output_node, weight))
    return genome



def generate_initial_population(population_size, genome_length):
    return [initialize_genome(genome_length) for _ in range(population_size)]

In [4]:
# Define the setup for the Echo State Network
def setup_esn(weight_matrix, input_dim=1, spectral_radius=0.9, ridge=1e-6):
    num_reservoir_nodes = weight_matrix.shape[0]
    # esn = nodes.Reservoir(
    #     Win=np.random.uniform(-0.5, 0.5, (num_reservoir_nodes, input_dim)),
    #     W=weight_matrix,
    #     bias=np.zeros((num_reservoir_nodes, 1)),
    #     spectral_radius=spectral_radius
    # ) >> nodes.Ridge(ridge=ridge)
    esn = nodes.Reservoir(
        Win=np.ones((num_reservoir_nodes, input_dim)),  # Input weight matrix
        W=weight_matrix,                                # Internal weight matrix
        bias=np.zeros((num_reservoir_nodes, 1))
    ) >> nodes.Ridge(ridge=ridge)                      # Readout layer with ridge regression
    return esn

# Initialize a population of genomes
def initialize_population(population_size, genome_length):
    return [np.random.uniform(-1, 1, genome_length) for _ in range(population_size)]

def mutate_genome(genome, use_hox=True):
    if use_hox:
        # Mutate using HOX based mutations
        return esf.mutate_genome_with_hox(genome)
    else:
        # Mutate without using HOX based mutations
        return esf.mutate_genome_without_hox(genome)

# Select the best-performing genomes
def select_best_genomes(population, fitness_scores, top_k=10):
    # Pair each genome with its fitness score
    paired_population_scores = list(zip(population, fitness_scores))
    
    # Sort the paired list by the fitness scores in descending order
    paired_population_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Select the top-k genomes
    selected_genomes = [genome for genome, score in paired_population_scores[:top_k]]
    
    return selected_genomes


# Reproduce genomes for the next generation
# def reproduce_genomes(selected_genomes, population_size, mutation_function=None):
#     new_population = []
#     while len(new_population) < population_size:
#         parent_genome = np.random.choice(selected_genomes)
#         if mutation_function:
#             child_genome = mutation_function(parent_genome)
#         else:
#             child_genome = parent_genome  # No mutation applied
#         new_population.append(child_genome)
#     return new_population

def reproduce_genomes(selected_genomes, population_size):
    new_population = []
    while len(new_population) < population_size:
        # Directly use random.choice for a pure Python selection instead of np.random.choice
        parent_genome = random.choice(selected_genomes)
        
        # Ensure mutate_genome creates a proper copy of the genome for mutation
        child_genome = mutate_genome(list(copy.deepcopy(parent_genome)), False)  # Ensure this is a deep copy if necessary
        
        new_population.append(child_genome)
    
    return new_population


# Visualization of performance scores
def visualize_performance(rmse_scores, mae_scores, mse_scores):
    plt.figure(figsize=(12, 6))
    plt.plot(rmse_scores, marker='o', linestyle='-', color='b', label='RMSE')
    plt.plot(mae_scores, marker='x', linestyle='-', color='r', label='MAE')
    plt.plot(mse_scores, marker='^', linestyle='-', color='g', label='MSE')
    plt.title('Performance Scores Over Generations')
    plt.xlabel('Generation')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.show()

# Visualization of weight matrices
def visualize_weight_matrices(best_weight_matrices):
    fig, axes = plt.subplots(len(best_weight_matrices), 1, figsize=(10, len(best_weight_matrices) * 5))
    for i, weight_matrix in enumerate(best_weight_matrices):
        ax = axes[i]
        img = ax.imshow(weight_matrix, cmap='viridis', aspect='auto')
        fig.colorbar(img, ax=ax)
        ax.set_title(f"Generation {i} Weight Matrix")
    plt.tight_layout()
    plt.show()


In [5]:
# Define the fitness function
def evaluate_genome_performance(genome, X_train, Y_train, X_test, Y_test):
    # Use the provided genome_transcription function to convert genome to weight matrix
    weight_matrix = gt.transcribe_hierarchical_genome_to_weight_matrix_old(genome)  # This function name may vary
    esn = setup_esn(weight_matrix)
    esn.fit(X_train, Y_train)
    predicted = esn.run(X_test)
    rmse = np.sqrt(mean_squared_error(Y_test, predicted))
    mae = mean_absolute_error(Y_test, predicted)
    mse = mean_squared_error(Y_test, predicted)
    return rmse, mae, mse

In [ ]:
fitness_scores = []
rmse_scores_over_generations = []
mae_scores_over_generations = []
mse_scores_over_generations = []
best_weight_matrices = []

# Evolution loop for "without mutation" experiment
def run_evolution_without_mutation():
    # Initialize population
    #population = initialize_population(population_size, genome_length)  # Use the correct initialization function
    population = [initialize_genome(genome_length) for _ in range(population_size)]
    
    for generation in range(n_generations):
        new_fitness_scores = []
        new_mae_scores = []
        new_mse_scores = []
        for i, genome in enumerate(population):
            
            #genome = mutate_genome(genome, False) 
            print(f"Genome: {genome}")
            genome = mutate_genome(genome, use_hox=False)
            print(f"Mutated Genome: {mutate_genome(genome, use_hox=False)}")
            
            # Evaluate genomes
            rmse, mae, mse = evaluate_genome_performance(genome, X_train, Y_train, X_test, Y_test)
            new_fitness_scores.append(rmse)
            new_mae_scores.append(mae)
            new_mse_scores.append(mse)
            
        fitness_scores = new_fitness_scores


        # Select best genomes
        #selected_genomes = select_best_genomes(population, fitness_scores)  # Implement this function
        best_indices = np.argsort(fitness_scores)[:10]
        selected_genomes = [population[i] for i in best_indices]
        
        # Reproduce genomes
        population = reproduce_genomes(selected_genomes, population_size)  # Implement this function
        
        rmse_scores_over_generations.append(min(fitness_scores))
        mae_scores_over_generations.append(min(new_mae_scores))
        mse_scores_over_generations.append(min(new_mse_scores))
        # Logging the best score for visualization
        print(f"Generation {generation}: Best RMSE Score: {min(fitness_scores)}, Best MAE Score: {min(new_mae_scores)}, Best MSE Score: {min(new_mse_scores)}")

        best_genome = selected_genomes[0]
        best_weight_matrix = gt.transcribe_hierarchical_genome_to_weight_matrix(best_genome)
        best_weight_matrices.append(best_weight_matrix)
    
        
    # Visualization of scores and weight matrices will be done after the loop
    visualize_performance(rmse_scores_over_generations, mae_scores_over_generations, mse_scores_over_generations)
    #visualize_weight_matrices(best_weight_matrices)


In [ ]:
def run_evolution_with_mutation():
    # Initialize population
    population = initialize_population(population_size, genome_length) # Use the correct initialization function
    for generation in range(n_generations):
        # Mutate and evaluate genomes
        for i, genome in enumerate(population):
            # Apply mutation using provided mutation function
            genome = mutate_genome(genome, True) # Use the correct mutation function
            # Evaluate the mutated genome
            rmse, mae, mse = evaluate_genome_performance(genome, X_train, Y_train, X_test, Y_test)
            population[i]['fitness'] = rmse  # Store fitness in the genome dict
        
    # Select best genomes
    selected_genomes = select_best_genomes(population)  # Implement this function
    
    # Reproduce genomes
    population = reproduce_genomes(selected_genomes, population_size)  # Implement this function
    
    # Logging the best score for visualization
    print(f"Generation {generation}: Best RMSE Score: {min([genome['fitness'] for genome in population])}")

# Visualization of scores and weight matrices will be done after the loop
visualize_performance(rmse_scores_over_generations, mae_scores_over_generations, mse_scores_over_generations)
visualize_weight_matrices(best_weight_matrices)

In [ ]:
#run_evolution_without_mutation()
#run_evolution_with_mutation()

In [6]:
fitness_scores = []
rmse_scores_over_generations = []
mae_scores_over_generations = []
mse_scores_over_generations = []
best_weight_matrices = []

# Evolution loop for "without mutation" experiment
def run_evolution_without_mutation_new():
    # Initialize population
    #population = initialize_population(population_size, genome_length)  # Use the correct initialization function
    #population = [initialize_genome(genome_length) for _ in range(population_size)]
    population = generate_initial_population(population_size, genome_length)
    
    
            
    for generation in range(n_generations):
        new_fitness_scores = []
        new_mae_scores = []
        new_mse_scores = []
        for i, genome_without_hox in enumerate(population):
            
            #genome = mutate_genome(genome, False) 
            # print(f"Genome: {genome}")
            # genome = mutate_genome(genome, use_hox=False)
            # print(f"Mutated Genome: {mutate_genome(genome, use_hox=False)}")
            
            # Evaluate genomes
            ###rmse, mae, mse = evaluate_genome_performance(genome, X_train, Y_train, X_test, Y_test)
            
            print(f"Genome: {genome_without_hox}")
            weight_matrix_without_hox = gt.transcribe_hierarchical_genome_to_weight_matrix_updated(genome_without_hox)
            weight_matrices_without_hox = [weight_matrix_without_hox]
            mutation_history_without_hox = ["initial"]
            
            
            genome_without_hox, mutation = esf.mutate_genome_without_hox(genome_without_hox)
            print(f"Mutated Genome: {genome_without_hox}")
            
            mutation_history_without_hox.append(mutation)
            weight_matrix_without_hox = gt.transcribe_hierarchical_genome_to_weight_matrix_updated(genome_without_hox)
            weight_matrices_without_hox.append(weight_matrix_without_hox)
            
            esn = setup_esn(weight_matrix_without_hox)
            esn.fit(X_train, Y_train)
            predicted = esn.run(X_test)
            rmse = np.sqrt(mean_squared_error(Y_test, predicted))
            mae = mean_absolute_error(Y_test, predicted)
            mse = mean_squared_error(Y_test, predicted)
    
            new_fitness_scores.append(rmse)
            new_mae_scores.append(mae)
            new_mse_scores.append(mse)
            
        fitness_scores = new_fitness_scores


        # Select best genomes
        #selected_genomes = select_best_genomes(population, fitness_scores)  # Implement this function
        best_indices = np.argsort(fitness_scores)[:10]
        selected_genomes = [population[i] for i in best_indices]
        
        # Reproduce genomes
        population = reproduce_genomes(selected_genomes, population_size)  # Implement this function
        
        rmse_scores_over_generations.append(min(fitness_scores))
        mae_scores_over_generations.append(min(new_mae_scores))
        mse_scores_over_generations.append(min(new_mse_scores))
        # Logging the best score for visualization
        print(f"Generation {generation}: Best RMSE Score: {min(fitness_scores)}, Best MAE Score: {min(new_mae_scores)}, Best MSE Score: {min(new_mse_scores)}")

        # best_genome = selected_genomes[0]
        # best_weight_matrix = gt.transcribe_hierarchical_genome_to_weight_matrix(best_genome)
        # best_weight_matrices.append(best_weight_matrix)
        """ fig, ax = plt.subplots(n_generations,1, figsize=(5,n_generations*5))

        for generation in range(n_generations):
            weight_matrix = weight_matrices_without_hox[generation]
            weight_matrix = np.abs(weight_matrix)

            if len(weight_matrix.shape) == 2:
                weight_matrix += weight_matrix != 0

                graph = nx.from_numpy_array(weight_matrix, create_using=nx.DiGraph)
                nx.draw_kamada_kawai(graph, ax=ax[generation], with_labels=True)
                ax[generation].set_title(f"Generation {generation} (without HOX), mutation {mutation_history_without_hox[generation]}")
        plt.show() """
    
        
    # Visualization of scores and weight matrices will be done after the loop
    visualize_performance(rmse_scores_over_generations, mae_scores_over_generations, mse_scores_over_generations)
    #visualize_weight_matrices(best_weight_matrices)
    
run_evolution_without_mutation_new()


Genome: [(8, 8, 0.16367425063332086), (3, 5, -0.14079869551125124), (8, 2, 0.9054889433338524), (5, 5, 0.03237711235319862), (6, 2, -0.604005699508783), (4, 2, -0.6460408756162652), (2, 9, 0.15328400318379742), (4, 5, 0.30727162381706474), (5, 8, -0.0237095377395109), (7, 6, -0.1268254525066188), (2, 5, -0.052780558361636754), (7, 4, -0.6644912604915583), (4, 6, -0.869556488733279), (4, 2, -0.7428849128697599), (3, 9, 0.6196913065495735), (8, 7, 0.6945615102782592), (4, 3, -0.5913988813849138), (4, 8, -0.7680662612270699), (9, 9, -0.08992184058761121), (7, 5, 0.3096593053846757), (1, 9, -0.47847869102298457), (2, 7, -0.9395486704503415), (5, 6, -0.7679571402981076), (2, 6, -0.05082568315801583), (6, 9, 0.35242594555802476), (2, 0, -0.338257438934757), (0, 3, -0.8360953947993233), (3, 9, 0.48653929533082363), (9, 3, -0.9020334167770232), (1, 5, -0.7444700712137027), (2, 1, -0.042881873768785495), (7, 0, 0.10487860563343121), (3, 1, -0.8874319659523213), (2, 4, 0.013560283440883003), (8,

ValueError: None value found after mutation

In [11]:
fitness_scores = []
rmse_scores_over_generations = []
mae_scores_over_generations = []
mse_scores_over_generations = []
best_weight_matrices = []

# Evolution loop for "with mutation" experiment
def run_evolution_with_mutation_new():
    # Initialize population
    #population = initialize_population(population_size, genome_length)  # Use the correct initialization function
    #population = [initialize_genome(genome_length) for _ in range(population_size)]
    population = generate_initial_population(population_size, genome_length)
    
    
            
    for generation in range(n_generations):
        new_fitness_scores = []
        new_mae_scores = []
        new_mse_scores = []
        for i, genome_with_hox in enumerate(population):
            
            #genome = mutate_genome(genome, False) 
            # print(f"Genome: {genome}")
            # genome = mutate_genome(genome, use_hox=False)
            # print(f"Mutated Genome: {mutate_genome(genome, use_hox=False)}")
            
            # Evaluate genomes
            ###rmse, mae, mse = evaluate_genome_performance(genome, X_train, Y_train, X_test, Y_test)
            
            print(f"Genome: {genome_with_hox}")
            print(f"Length of Genome: {len(genome_with_hox)}")
            weight_matrix_with_hox = gt.transcribe_hierarchical_genome_to_weight_matrix_updated(genome_with_hox)
            weight_matrices_with_hox = [weight_matrix_with_hox]
            mutation_history_with_hox = ["initial"]
            
            
            mutated_genome_with_hox, mutation = esf.mutate_genome_with_hox(genome_with_hox)
            if(len(genome_with_hox) < 1):
                mutated_genome_with_hox = genome_with_hox
                
            print(f"Mutated Genome: {mutated_genome_with_hox}")
            print(f"Length of Mutated Genome: {len(mutated_genome_with_hox)}")
            mutation_history_with_hox.append(mutation)
            weight_matrix_with_hox = gt.transcribe_hierarchical_genome_to_weight_matrix_updated(mutated_genome_with_hox)
            weight_matrices_with_hox.append(weight_matrix_with_hox)
            
            esn = setup_esn(weight_matrix_with_hox)
            esn.fit(X_train, Y_train)
            predicted = esn.run(X_test)
            rmse = np.sqrt(mean_squared_error(Y_test, predicted))
            mae = mean_absolute_error(Y_test, predicted)
            mse = mean_squared_error(Y_test, predicted)
    
            new_fitness_scores.append(rmse)
            new_mae_scores.append(mae)
            new_mse_scores.append(mse)
            
        fitness_scores = new_fitness_scores


        # Select best genomes
        #selected_genomes = select_best_genomes(population, fitness_scores)  # Implement this function
        best_indices = np.argsort(fitness_scores)[:10]
        selected_genomes = [population[i] for i in best_indices]
        
        # Reproduce genomes
        population = reproduce_genomes(selected_genomes, population_size)  # Implement this function
        
        rmse_scores_over_generations.append(min(fitness_scores))
        mae_scores_over_generations.append(min(new_mae_scores))
        mse_scores_over_generations.append(min(new_mse_scores))
        # Logging the best score for visualization
        print(f"Generation {generation}: Best RMSE Score: {min(fitness_scores)}, Best MAE Score: {min(new_mae_scores)}, Best MSE Score: {min(new_mse_scores)}")

        # best_genome = selected_genomes[0]
        # best_weight_matrix = gt.transcribe_hierarchical_genome_to_weight_matrix(best_genome)
        # best_weight_matrices.append(best_weight_matrix)
        """ fig, ax = plt.subplots(n_generations,1, figsize=(5,n_generations*5))

        for generation in range(n_generations):
            weight_matrix = weight_matrices_without_hox[generation]
            weight_matrix = np.abs(weight_matrix)

            if len(weight_matrix.shape) == 2:
                weight_matrix += weight_matrix != 0

                graph = nx.from_numpy_array(weight_matrix, create_using=nx.DiGraph)
                nx.draw_kamada_kawai(graph, ax=ax[generation], with_labels=True)
                ax[generation].set_title(f"Generation {generation} (without HOX), mutation {mutation_history_without_hox[generation]}")
        plt.show() """
    
        
    # Visualization of scores and weight matrices will be done after the loop
    visualize_performance(rmse_scores_over_generations, mae_scores_over_generations, mse_scores_over_generations)
    #visualize_weight_matrices(best_weight_matrices)
    
run_evolution_with_mutation_new()


Genome: [(3, 7, -0.6312240204746027), (8, 7, -0.3483198153915743), (5, 9, -0.5952879920216276), (4, 3, 0.31079883080228266), (1, 1, 0.6944005791787491), (8, 1, 0.42138879307191224), (5, 2, -0.06804013582596347), (8, 9, 0.48403778266131114), (9, 3, 0.8387378503020588), (0, 0, -0.822625079611562), (1, 2, -0.9602102901366338), (6, 7, -0.7049324658730645), (0, 1, -0.32292145866527044), (5, 2, -0.8041772531256413), (5, 0, 0.822998472124107), (8, 3, -0.5169101435417336), (5, 3, -0.015103639647151557), (0, 1, -0.5669541262086788), (0, 4, 0.20548168831131464), (0, 7, 0.20651924054346216), (1, 6, -0.721005913518245), (4, 1, -0.14416902943518717), (6, 8, -0.9721223017651905), (4, 7, 0.8996256388507677), (4, 8, 0.41364232971634696), (6, 1, 0.5517125555638258), (1, 9, -0.46499541987672743), (3, 8, 0.12573917376218025), (6, 8, -0.9801010154844791), (0, 4, 0.9683537049484914), (7, 1, -0.7055000533081404), (1, 1, 0.004055589974230944), (7, 0, 0.3117247063031674), (0, 4, 0.49039465100600865), (1, 0, -

ValueError: Invalid genome structure

In [12]:

print(f"Length of Mutated Genome: {len(mutated_genome_with_hox)}")

NameError: name 'genome_with_hox' is not defined